<a href="https://colab.research.google.com/github/ShashankKamath/DecisionTree_ML/blob/master/DT_q2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import files
uploaded = files.upload()


Saving hw2_extra.csv to hw2_extra (1).csv
Saving hw2_question1.csv to hw2_question1.csv


In [0]:
import pandas as pd
import numpy as np
data = pd.read_csv("hw2_extra.csv")

# header=[]
# data.columns=header

In [12]:
data.head()

,Temperature,Humidity,Sky_condition,Output
0,hot,high,cloudy,1
1,hot,high,cloudy,1
2,hot,high,cloudy,1
3,hot,high,cloudy,1
4,hot,high,cloudy,1


In [0]:
data=data.values.tolist()

In [0]:
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

In [0]:
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

In [0]:
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [0]:
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

In [0]:
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [0]:
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

In [0]:
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

In [0]:
import pandas as pd
import numpy as np
data = pd.read_csv("hw2_question1.csv")
header=["Thickness", "Cellsize", "Cellshape", "Adhesion","Ecellsize","Nuclei","Chromatin","Nucleoli","Mitosis","Class"]
data.columns=header
data = data.sample(frac=1).reset_index(drop=True)
train_data=pd.DataFrame()
test_data=pd.DataFrame()
class_2_count=[295,148]
class_4_count=[159,80]
for i in range(0,data.shape[0]):
    if(data.loc[i,'Class']==2 and class_2_count[0]<=295 and class_2_count[0]>0):
        train_data=train_data.append(data.iloc[i])
        class_2_count[0]-=1
    if(data.loc[i,'Class']==4 and class_4_count[0]<=159 and class_4_count[0]>0):
        train_data=train_data.append(data.iloc[i])
        class_4_count[0]-=1
    if(data.loc[i,'Class']==2 and class_2_count[1]<=148 and class_2_count[1]>0):
        test_data=test_data.append(data.iloc[i])
        class_2_count[1]-=1
    if(data.loc[i,'Class']==4 and class_4_count[1]<=80 and class_4_count[1]>0):
        test_data=test_data.append(data.iloc[i])
        class_4_count[1]-=1
train_data=train_data[header]
test_data=test_data[header]

train_set=train_data.values.tolist()
test_set=test_data.values.tolist()

In [30]:
tree=build_tree(train_set,9,1)
print_tree(tree)

[X2 < 3.000]
 [X1 < 7.000]
  [X6 < 7.000]
   [X6 < 5.000]
    [X1 < 4.000]
     [X1 < 2.000]
      [X1 < 1.000]
       [2.0]
       [2.0]
      [X1 < 2.000]
       [2.0]
       [2.0]
     [X1 < 4.000]
      [2.0]
      [2.0]
    [X5 < 2.000]
     [4.0]
     [X1 < 1.000]
      [2.0]
      [2.0]
   [X1 < 4.000]
    [2.0]
    [X1 < 5.000]
     [4.0]
     [X1 < 5.000]
      [4.0]
      [4.0]
  [X2 < 2.000]
   [2.0]
   [X1 < 10.000]
    [X1 < 7.000]
     [4.0]
     [4.0]
    [4.0]
 [X3 < 3.000]
  [X7 < 4.000]
   [X1 < 5.000]
    [X1 < 2.000]
     [2.0]
     [X1 < 2.000]
      [2.0]
      [2.0]
    [X1 < 5.000]
     [2.0]
     [2.0]
   [X1 < 5.000]
    [2.0]
    [X1 < 7.000]
     [X1 < 6.000]
      [4.0]
      [X1 < 6.000]
       [4.0]
       [4.0]
     [X1 < 7.000]
      [4.0]
      [4.0]
  [X2 < 5.000]
   [X6 < 3.000]
    [X4 < 4.000]
     [X1 < 8.000]
      [X1 < 6.000]
       [X1 < 5.000]
        [2.0]
        [X1 < 5.000]
         [2.0]
         [2.0]
       [2.0]
      [2.0]
     [4.0]